## 🛠️ **ETL (Extract, Transform, Load)**
### **📂Procesamos el 2do archivo: `user_items.json.gz`**


📂 **user_reviews.json**:: 
- 🗃️ `Items `**:** Desanidado: era una lista de diccionarios
- 🔄 `Playtime_forever `: Transformamos los minutos a horas

#### 📦 **EXTRACT**

#### **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
from data_utils import data_type_check, duplicados_columna
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

#### **Descomprimimos el archivo gz** 

In [2]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/users_items.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)


Archivo descomprimido: ../0 Dataset/users_items.json


### **📂Procesamos el 2do archivo: `user_items.json.gz`**


- Tomamos los datos del archivo JSON, transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [3]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("../0 Dataset/users_items.json", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_users_items = pd.DataFrame(rows)
#Veamos unos registros al asar
df_users_items.sample(2)

,user_id,items_count,steam_id,user_url,items
51672,76561198044595704,15,76561198044595704,http://steamcommunity.com/profiles/76561198044...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
12316,richardislongfor,20,76561198047947089,http://steamcommunity.com/id/richardislongfor,"[{'item_id': '6060', 'item_name': 'STAR WARS™ ..."


In [4]:
# Usando una funcion personalizada vemos las variables categóricas, numéricas, dimensiones del dataframe, nulos, tipos de datos   
data_type_check(df_users_items)


 Resumen del Dataframe 
Dimensiones:  (88310, 5)
       columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0      user_id       100.0      0.0            0    object
1  items_count       100.0      0.0            0     int64
2     steam_id       100.0      0.0            0    object
3     user_url       100.0      0.0            0    object
4        items       100.0      0.0            0    object


### 🔁 **TRANSFORM**

In [ ]:
#Eliminamos las columnas irrelevantes que no nos riven
df_users_items.drop(["user_url","items"], axis=1, inplace=True)

🗂️ Desanidado de la Columna **'items'** (es una lista de diccionarios)


In [5]:
#Creamos una nueva fila para cada elemento de la lista en la columna items
df_users_items = df_users_items.explode("items").reset_index()
#Eliminamos la columna index
df_users_items = df_users_items.drop(columns="index")
# Creamos una nueva columna para cada elemento de la lista en la columna items
df_users_items = pd.concat([df_users_items, pd.json_normalize(df_users_items['items'])], axis=1)

In [6]:
# Usando una funcion personalizada vemos las variables categóricas, numéricas, dimensiones del dataframe, nulos, tipos de datos   
data_type_check(df_users_items)


 Resumen del Dataframe 


Dimensiones:  (5170015, 9)
            columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0           user_id      100.00     0.00            0    object
1       items_count      100.00     0.00            0     int64
2          steam_id      100.00     0.00            0    object
3          user_url      100.00     0.00            0    object
4             items       99.67     0.33        16806    object
5           item_id       99.67     0.33        16806    object
6         item_name       99.67     0.33        16806    object
7  playtime_forever       99.67     0.33        16806   float64
8   playtime_2weeks       99.67     0.33        16806   float64


Playtime_forever: Transformamos los minutos a horas 

In [8]:
'''
playtime_forever
'''
#borramos donde no haya tiempo de juego
df_users_items = df_users_items[df_users_items['playtime_forever'] != 0]
#Convertimos los minutos a horas
df_users_items['playtime_forever'] = df_users_items['playtime_forever'] / 60

'''
User_id:
'''

# Eliminar espacios en blanco al principio y al final de los valores
df_users_items['user_id'] = df_users_items['user_id'].str.strip()
# Llenar valores nulos con un valor específico (por ejemplo, cadena vacía)
df_users_items['user_id'].fillna('', inplace=True)
# Eliminar caracteres especiales o no imprimibles
df_users_items['user_id'] = df_users_items['user_id'].str.replace(r'\W', '')
# Convertir a tipo string
df_users_items = df_users_items.astype({'user_id': 'string'})


#### **item_id**

In [ ]:
df_users_items

In [19]:
df_users_items['item_id'] = df_users_items['item_id'].fillna(-1)
# Remove empty strings (if needed)
df_users_items['item_id'] = df_users_items['item_id'].str.replace('"', '')



ValueError: invalid literal for int() with base 10: 'nan'

In [17]:



# Call your data_type_check function here
data_type_check(df_users_items)

ValueError: invalid literal for int() with base 10: 'nan'

### 📤 **LOAD**

In [ ]:
#convertimos a csv
items= df_users_items.to_csv('./0 Dataset/user_items_LIMPIO.csv',index=False) 
# Leemos el archivo CSV en un DataFrame de pandas
items = pd.read_csv('./0 Dataset/user_items_LIMPIO.csv')

tengo esto dos veces+

In [ ]:
# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(items)
# Escribir la tabla en un archivo Parquet
pq.write_table(table, './0 Dataset/user_items_LISTO.parquet')

In [ ]:
# Leer el archivo Parquet en una tabla de PyArrow
table = pq.read_table('./0 Dataset/user_items_LISTO.parquet')
# Convertir la tabla de PyArrow a un DataFrame de pandas
items_parquet = table.to_pandas()
#Borramos el csv limpio ya que nos sirve mas en parquet
os.remove('./0 Dataset/user_items_LIMPIO.csv')
#Dejamos informacion de nuestro dataset
data_type_check(df_users_items)


 Resumen del Dataframe 
Dimensiones:  (3285246, 7)
            columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0           user_id       100.0      0.0            0  string[python]
1       items_count       100.0      0.0            0           int64
2          steam_id       100.0      0.0            0          object
3           item_id       100.0      0.0            0           int32
4         item_name       100.0      0.0            0          object
5  playtime_forever       100.0      0.0            0         float64
6   playtime_2weeks       100.0      0.0            0         float64


## **📂Procesamos el 3er archivo: `user_reviews.json.gz`**


### 📦 **EXTRACT**

In [ ]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("./0 Dataset/user_reviews.json", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_user_reviews = pd.DataFrame(rows)
#Veamos unos registros al asar
df_user_reviews.sample(2)

,user_id,user_url,reviews
14316,76561198064936176,http://steamcommunity.com/profiles/76561198064...,"[{'funny': '', 'posted': 'Posted January 23, 2..."
9185,terralark,http://steamcommunity.com/id/terralark,"[{'funny': '', 'posted': 'Posted October 27, 2..."



 Resumen del Dataframe 
Dimensiones:  (25799, 3)
    columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0   user_id       100.0      0.0            0    object
1  user_url       100.0      0.0            0    object
2   reviews       100.0      0.0            0    object


,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


from the creaters of the walking dead, i present to you, the wolf among us. a twisted an unhappy place where crimes are made in the town of the fables. Ecperience one of the most mind-bending, jaw-dropping twists you will see in all of gaming history...well, some of gaming history. SPOILERS: i really liked how bigby turns into his true form to fight all the bloody marys... its just like when neo fought all the agent smiths in the matrix. what i didnt really like is when i chose to lock the crooked man up it glitches my game and i start from the near begining where bigby fought the crooked man's crew, exept the guy who runs a strip club wasnt there. so i was really confused. but anyway i really recommend this game if you want twists and ♥♥♥♥ed up scenes.
this game is awesome,this game is ♥♥♥♥ed up and this game is so sad and depressing. if you want these types of games i would strongly reccomend youto buy it. its worth your money
----------------------------------------
from the creater

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


''


 Resumen del Dataframe 
Dimensiones:  (57397, 9)
               columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0              user_id      100.00     0.00            0    object
1             user_url      100.00     0.00            0    object
2        reviews_funny       13.76    86.24        49498    object
3       reviews_posted      100.00     0.00            0    object
4  reviews_last_edited       10.28    89.72        51499    object
5      reviews_item_id      100.00     0.00            0    object
6      reviews_helpful      100.00     0.00            0    object
7    reviews_recommend      100.00     0.00            0      bool
8       reviews_review       99.95     0.05           30    object
 
Vemos que todavia hay faltantes:
     reviews_funny:                   %86.24
     reviews_last_edited              %89.72
Procedemos a eliminarlos:


Observamos las columnas que nos quedaron: 
 

 Resumen del Dataframe 
Dimensiones:  (57397, 7)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            user_id      100.00     0.00            0    object
1           user_url      100.00     0.00            0    object
2     reviews_posted      100.00     0.00            0    object
3    reviews_item_id      100.00     0.00            0    object
4    reviews_helpful      100.00     0.00            0    object
5  reviews_recommend      100.00     0.00            0      bool
6     reviews_review       99.95     0.05           30    object



 Resumen del Dataframe 
Dimensiones:  (47626, 7)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            user_id      100.00     0.00            0    object
1           user_url      100.00     0.00            0    object
2     reviews_posted      100.00     0.00            0     int32
3    reviews_item_id      100.00     0.00            0    object
4    reviews_helpful      100.00     0.00            0    object
5  reviews_recommend      100.00     0.00            0      bool
6     reviews_review       99.94     0.06           27    object



 Resumen del Dataframe 
Dimensiones:  (47626, 7)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            user_id      100.00     0.00            0    object
1           user_url      100.00     0.00            0    object
2     reviews_posted      100.00     0.00            0     int32
3    reviews_item_id      100.00     0.00            0    object
4    reviews_helpful      100.00     0.00            0    object
5  reviews_recommend      100.00     0.00            0      bool
6     reviews_review       99.94     0.06           27    object


### 📤 **LOAD**

Eliminamos el archivo descomprimidos que ahora tenemos limpio y liviano en formato parquet

In [ ]:
# Lista de archivos descomprimidos
gz_descomprimidos = [
    './0 Dataset/users_items.json'
]

# Eliminar archivos descomprimidos
for archivo_json in gz_descomprimidos:
    try:
        os.remove(archivo_json)
        print(f'Archivo eliminado: {archivo_json}')
    except FileNotFoundError:
        print(f'Archivo no encontrado: {archivo_json}')

Archivo eliminado: ./0 Dataset/steam_games.json
Archivo eliminado: ./0 Dataset/user_reviews.json
Archivo eliminado: ./0 Dataset/users_items.json


Observamos el tiempo de ejecucion total de nuestro proceso ETL 🔥

In [ ]:
# Obtener el tiempo de finalización
end_time = time.time()
# Calcular el tiempo total de ejecución
total_time = end_time - start_time
# Convertir a minutos y redondear a 2 decimales
total_time_minutes = round(total_time / 60, 2)
# Imprimir resultados
print(f"Tiempo total de ejecución de este ipynb: {total_time_minutes} minutos")

Tiempo total de ejecución de este ipynb: 3.79 minutos
